We will import and clean a variety of data sources provided by New York City Public Schools. We will them explore correlations and visualizations to see if there are any notable relationships in the data.

In [1]:
import pandas as pd
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]
data = {}

# Read data from csv files into a dictionary for future reference
for filename in data_files:
    df = pd.read_csv("schools/"+filename)
    data[filename.split(".")[0]] = df

We have read each file into a pandas dataframe, and then stored all of the dataframes in a dictionary. This will give us a convenient way to store them, and a quick way to reference them in the future.

In [4]:
data.keys()

dict_keys(['ap_2010', 'class_size', 'demographics', 'graduation', 'hs_directory', 'sat_results'])

Let's explore sat_results to see what we can discover. Exploring the dataframe will help us understand the structure of the data, and make it easier for us to analyze it.

In [5]:
data["sat_results"].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


The DBN appears to be a unique ID for each school. Let's explore all the dataframes.

In [10]:
for key in data.keys():
    print(data[key].head())

      DBN                             SchoolName AP Test Takers   \
0  01M448           UNIVERSITY NEIGHBORHOOD H.S.              39   
1  01M450                 EAST SIDE COMMUNITY HS              19   
2  01M515                    LOWER EASTSIDE PREP              24   
3  01M539         NEW EXPLORATIONS SCI,TECH,MATH             255   
4  02M296  High School of Hospitality Management               s   

  Total Exams Taken Number of Exams with scores 3 4 or 5  
0                49                                   10  
1                21                                    s  
2                26                                   24  
3               377                                  191  
4                 s                                    s  
   CSD BOROUGH SCHOOL CODE                SCHOOL NAME GRADE  PROGRAM TYPE  \
0    1       M        M015  P.S. 015 Roberto Clemente     0K       GEN ED   
1    1       M        M015  P.S. 015 Roberto Clemente     0K          CTT   
2    1

Each data set appears to either have a DBN column, or the information we need to create one. Some fields look interesting for mapping -- particularly Location 1, which contains coordinates inside a larger string. Some of the data sets appear to contain multiple rows for each school (because the rows have duplicate DBN values). That means we’ll have to do some preprocessing to ensure that each DBN is unique within each data set.

Now let's read in survey_all.txt and survey_d75.txt, using the keyword argument encoding="windows-1252", as we know these files are encoded in that format. Then we can combine the data into a single dataframe by using the pandas concat() function along axis 0.

In [12]:
all_survey = pd.read_csv("schools/survey_all.txt", delimiter="\t", encoding = "windows-1252")
d75_survey = pd.read_csv("schools/survey_d75.txt", delimiter="\t", encoding = "windows-1252")
survey = pd.concat([all_survey, d75_survey], axis=0, sort=False)

survey.head()

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,...,s_q14_2,s_q14_3,s_q14_4,s_q14_5,s_q14_6,s_q14_7,s_q14_8,s_q14_9,s_q14_10,s_q14_11
0,01M015,M015,P.S. 015 Roberto Clemente,0,No,0.0,Elementary School,NaN,88,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M019,M019,P.S. 019 Asher Levy,0,No,0.0,Elementary School,NaN,100,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M020,M020,P.S. 020 Anna Silver,0,No,0.0,Elementary School,NaN,88,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01M034,M034,P.S. 034 Franklin D. Roosevelt,0,Yes,0.0,Elementary / Middle School,89.0,73,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01M063,M063,P.S. 063 William McKinley,0,No,0.0,Elementary School,NaN,100,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
survey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1702 entries, 0 to 55
Columns: 2773 entries, dbn to s_q14_11
dtypes: float64(2762), int64(6), object(5)
memory usage: 36.0+ MB


In [30]:
survey.isnull().sum().value_counts().sort_index(ascending=False).head(20)

1702     46
1662      5
1648    163
1646    650
1200     18
1198     11
1161      5
1155      4
721       5
720     392
666     157
661       1
547       1
72        5
64        2
63       57
62      248
60        1
59        2
58      242
dtype: int64

There are over 2700 columns, with many hundreds of columns that have no data for a large percentage of rows so we can drop them. Most importantly, the survey data has a dbn column which we can use for indexing and matching with our other dataframes.

These are the relevant columns which give us aggregate survey data about how parents, teachers, and students feel about school safety, academic performance, and more that we will keep: ["dbn", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]

In [31]:
survey["DBN"]=survey["dbn"]
columns = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]
survey = survey.loc[:,columns]
data["survey"] = survey
data["survey"].shape

(1702, 23)

In our dataset, class_size doesn't appear to have the column at all. Here are the first few rows of the data set:

In [35]:
data["class_size"].head(5)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN


Here are the first few rows of the sat_results data, which does have a DBN column:

In [37]:
data["sat_results"].head(10)

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384
5,01M515,LOWER EAST SIDE PREPARATORY HIGH SCHOOL,112,332,557,316
6,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159,522,574,525
7,01M650,CASCADES HIGH SCHOOL,18,417,418,411
8,01M696,BARD HIGH SCHOOL EARLY COLLEGE,130,624,604,628
9,02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,16,395,400,387


Based on the above, it seems that the DBN in the sat_results data is just a combination of the CSD and SCHOOL CODE columns in the class_size data. The main difference is that the DBN is padded, so that the CSD portion of it always consists of two digits. 

For example, LOWER EAST SIDE PREPARATORY HIGH SCHOOL has a DBN of 01M515, which is most likely what P.S. 015 Roberto Clemente School would have too if we combined a padded CSD value with SCHOOL CODE.

Let's create the DBN column per the guideline above.

In [38]:
data["hs_directory"]["DBN"] = data["hs_directory"]["dbn"]

# custom function that takes in a number.
# Converts the number to a string using the str() function.
# Check the length of the string using the len() function.
# If the string is two digits long, returns the string.
# If the string is one digit long, adds a 0 to the front of the string, then returns it.

def pad_csd(x):
    str_x = str(x)
    return str_x.zfill(2)

data["class_size"]["padded_csd"] = data["class_size"]["CSD"].apply(pad_csd)

data["class_size"]["DBN"] = data["class_size"]["padded_csd"] + data["class_size"]["SCHOOL CODE"] 

data["class_size"]["DBN"].tail(10)

27601    32K564
27602    32K564
27603    32K564
27604    32K564
27605    32K564
27606    32K564
27607    32K564
27608    32K564
27609    32K564
27610    32K564
Name: DBN, dtype: object

Now we're almost ready to combine our data sets. Before we do, let's take some time to calculate variables that will be useful in our analysis. Let's create a column that totals up the SAT scores for the different sections of the exam. This will make it much easier to correlate scores with demographic factors because we'll be working with a single number, rather than three different ones.

In [40]:
convert_list = ["SAT Math Avg. Score", "SAT Critical Reading Avg. Score", "SAT Writing Avg. Score"]
for col_mame in convert_list:
    data["sat_results"][col_mame] = pd.to_numeric(data["sat_results"][col_mame], errors="coerce")

data["sat_results"]["sat_score"] = data["sat_results"]["SAT Math Avg. Score"] + data["sat_results"]["SAT Critical Reading Avg. Score"] + data["sat_results"]["SAT Writing Avg. Score"]

data["sat_results"]["sat_score"].head(10)

0    1122.0
1    1172.0
2    1149.0
3    1174.0
4    1207.0
5    1205.0
6    1621.0
7    1246.0
8    1856.0
9    1182.0
Name: sat_score, dtype: float64

Next, we'll want to parse the latitude and longitude coordinates for each school. This will enable us to map the schools and uncover any geographic patterns in the data. The coordinates are currently in the text field Location 1 in the hs_directory data set.

In [41]:
import re
def get_latitude (data):
    latlong = re.findall("(?<=\().+(?=\))",data)
    return (str(latlong[0]).split(", ")[0])
    
data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(get_latitude)

data["hs_directory"]["lat"].head()

0     40.67029890700047
1      40.8276026690005
2    40.842414068000494
3     40.71067947100045
4    40.718810094000446
Name: lat, dtype: object

In [42]:
def get_longitude (data):
    latlong = re.findall("(?<=\().+(?=\))",data)
    return (str(latlong[0]).split(", ")[1])
    
data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(get_longitude)

data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"], errors="coerce")
data["hs_directory"]["lon"] = pd.to_numeric(data["hs_directory"]["lon"], errors="coerce")

data["hs_directory"][["DBN","lat","lon"]].head()

,DBN,lat,lon
0,17K548,40.670299,-73.961648
1,09X543,40.827603,-73.904475
2,09X327,40.842414,-73.916162
3,02M280,40.710679,-74.000807
4,28Q680,40.718810,-73.806500


In our dataset, class_size, graduation, and demographics have duplicate DBN values. For example there are over 26,000 rows with at least one duplicate in DBN for class_size:

In [47]:
data["class_size"]["DBN"].duplicated().value_counts()

True     26124
False     1487
Name: DBN, dtype: int64

In [48]:
data["class_size"][data["class_size"]["DBN"].duplicated()].head(10)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01,01M015
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN,01,01M015
5,1,M,M015,P.S. 015 Roberto Clemente,02,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
6,1,M,M015,P.S. 015 Roberto Clemente,03,GEN ED,-,-,-,12.0,1.0,12.0,12.0,12.0,ATS,NaN,01,01M015
7,1,M,M015,P.S. 015 Roberto Clemente,03,CTT,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN,01,01M015
8,1,M,M015,P.S. 015 Roberto Clemente,04,GEN ED,-,-,-,26.0,2.0,13.0,12.0,14.0,ATS,NaN,01,01M015
9,1,M,M015,P.S. 015 Roberto Clemente,05,GEN ED,-,-,-,27.0,1.0,27.0,27.0,27.0,ATS,NaN,01,01M015
10,1,M,M015,P.S. 015 Roberto Clemente,0K-09,SPEC ED,-,-,12:1:1,9.0,1.0,9.0,9.0,9.0,ATS,NaN,01,01M015


Seems like the data is based on the "GRADE " column. Let's explore the values that this column has, as we are only interested in high schools.

In [57]:
data["class_size"]["GRADE "].value_counts()

09-12      10644
MS Core     4762
0K-09       1384
0K          1237
01          1185
02          1167
03          1143
04          1140
05          1086
06           846
07           778
08           735
09            20
Name: GRADE , dtype: int64

Because we're dealing with high schools, we're only concerned with grades 9 through 12. That means we only want to pick rows where the value in the GRADE column is 09-12.

If we look at the values for PROGRAM TYPE, we get the following:

In [62]:
data["class_size"]["PROGRAM TYPE"].value_counts(normalize=True)

GEN ED     0.556704
CTT        0.285528
SPEC ED    0.139817
G&T        0.017951
Name: PROGRAM TYPE, dtype: float64

Each school can have multiple program types. Because GEN ED is the largest category by far (>55%), let's only select rows where PROGRAM TYPE is GEN ED.